In [ ]:

from gensim import models

In [ ]:
sentences, cur_sent = list(), list()
with open('wackypedia_en1.words10.20Mwords', errors='ignore') as f:
 for line in f:
 line = line.strip()
 if line == '</s>':
 sentences.append(cur_sent)
 cur_sent = list()
 elif line != '<s>' and not line.startswith('<text') and not line.startswith('</text'):
 cur_sent.append(line.split('\t')[0])


In [ ]:
model_big_ten = gensim.models.Word2Vec(sentences,min_count=5,window=1,size=10)
model_small_ten = gensim.models.Word2Vec(sentences,min_count=5,window=10,size=10)

model_big_five = gensim.models.Word2Vec(sentences,min_count=5,window=1,size=500)
model_small_five = gensim.models.Word2Vec(sentences,min_count=5,window=10,size=500)